In [1]:
import pandas as pd, numpy as np

In [2]:
# merge hadicappers' history bets into one dataframe
names = [
    'Smart',
    'Bartley',
    'Aronson',
    'Burns',
    'Barone', # not in Ji's original list
    'Bitler',
    'Power',
    'Ross',
    'Diamond',
    'Trapp',
    'Sports',
    'Eddie',
    'Schule',
    'DAmico',
    'Duffy',
    'Thomas',
    'Hunter',
    'Compeau', # not in Ji's original list
    'Syndicate',
    'Lundin',
    'Simulator',
    'Wilson',
    'Monohan',
    'Vinceletti',
    'Rickenbach',
    'Higgs',
    'Nover',
    'Brown',
    'Karpinski',
    'Rogers'
    ]

df = pd.DataFrame()
for i in range(1,31): 
    newdf = pd.read_pickle('HC' + str(i) + '.gz', compression='gzip')
    
    # if you want the last name of the HC, switch to use names array
    #newdf['Handicapper'] = names[i-1]
    newdf['Handicapper'] = 'HC'+str(i)
    df = df.append(newdf, ignore_index=True)

In [3]:
# it looks like there's duplicates in the data, so we will take the first instance
ml_bets_all_hcs = df[df['Bet']=='MONEY LINE'].pivot_table(values=['MLBet'], 
                                                          columns='Handicapper', 
                                                          index=['GameKey'], aggfunc='first')

# drop multiindex level 0
ml_bets_all_hcs.columns = ml_bets_all_hcs.columns.droplevel(0)

#ml_bets_all_hcs.head()

In [4]:
ml_juice_all_hcs = df[df['Bet']=='MONEY LINE'].pivot_table(values=['Juice'], 
                                                           columns='MLBet', 
                                                           index=['GameKey'], aggfunc='max')

ml_juice_all_hcs.columns = ml_juice_all_hcs.columns.droplevel(0)
ml_juice_all_hcs.columns.name = 'Juice'
ml_juice_all_hcs.columns = ['Juice_Away', 'Juice_Home']
#ml_juice_all_hcs.head()

In [5]:
ml_game_info = df[df['Bet']=='MONEY LINE'].pivot_table(values=['GameTime', 'League','Home', 'Away', 'MLWinner'], 
                                                       index=['GameKey'], aggfunc='first')
#ml_game_info.head()

In [6]:
columns = ['League', 'GameTime', 'Away', 'Juice_Away', 'Home', 'Juice_Home', 'MLWinner',
           'HC1', 'HC2', 'HC3', 'HC4', 'HC5', 'HC6', 'HC7', 'HC8', 'HC9', 'HC10',
           'HC11', 'HC12', 'HC13', 'HC14', 'HC15', 'HC16', 'HC17', 'HC18', 'HC19', 'HC20', 
           'HC21', 'HC22', 'HC23', 'HC24', 'HC25', 'HC26', 'HC27', 'HC28', 'HC29', 'HC30']
ml_bets = ml_game_info.join(ml_juice_all_hcs).join(ml_bets_all_hcs)[columns].sort_values(by=['GameTime'])

In [7]:
# if juice is empty, put in 100 
ml_bets.Juice_Away.fillna(100, inplace=True)
ml_bets.Juice_Home.fillna(100, inplace=True)

# conver juice to int64
ml_bets.Juice_Away = pd.to_numeric(ml_bets.Juice_Away)
ml_bets.Juice_Home = pd.to_numeric(ml_bets.Juice_Home)

#ml_bets.dtypes

In [8]:
for each in ml_bets.League.unique():
    print(ml_bets[ml_bets['League']==each]['League'].groupby([ml_bets.GameTime.dt.year, ml_bets.League]).size(), "\n")


GameTime  League
2013      MLB        657
2014      MLB       1952
2015      MLB       2086
2016      MLB       2151
2017      MLB       2241
2018      MLB        165
Name: League, dtype: int64 

GameTime  League
2013      NFL        39
2014      NFL        81
2015      NFL        80
2016      NFL       102
2017      NFL        92
2018      NFL         4
Name: League, dtype: int64 

GameTime  League
2013      NCAAF     23
2014      NCAAF     70
2015      NCAAF     58
2016      NCAAF     61
2017      NCAAF     65
2018      NCAAF      4
Name: League, dtype: int64 

GameTime  League
2013      NHL        169
2014      NHL        640
2015      NHL        850
2016      NHL        966
2017      NHL       1069
2018      NHL        534
Name: League, dtype: int64 

GameTime  League
2013      NBA         6
2014      NBA       104
2015      NBA       129
2016      NBA       174
2017      NBA       169
2018      NBA       109
Name: League, dtype: int64 

GameTime  League
2013      NCAAB       7
201

In [9]:
mlb_ml_bets = ml_bets[(ml_bets['League']=='MLB') & (ml_bets['MLWinner'] !='')]

mlb_ml_bets.shape

(9173, 37)

#### Model 1 - Exclude home and away team information, just use handicapper bets

In [10]:
juice_bins = [-9999, -300, -200, -190, -180, -170, -160, -150, -140, -130, -120, -110, -100, 
             100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 300, 9999]

make_dummies = ['Binned_Juice_Away', 'Binned_Juice_Home', 
           'HC1', 'HC2', 'HC3', 'HC4', 'HC5', 'HC6', 'HC7', 'HC8', 'HC9', 'HC10',
           'HC11', 'HC12', 'HC13', 'HC14', 'HC15', 'HC16', 'HC17', 'HC18', 'HC19', 'HC20', 
           'HC21', 'HC22', 'HC23', 'HC24', 'HC25', 'HC26', 'HC27', 'HC28', 'HC29', 'HC30']

mlb_ml_bets['Binned_Juice_Away'] = pd.cut(mlb_ml_bets.Juice_Away, bins=juice_bins, labels=False)
mlb_ml_bets['Binned_Juice_Home'] = pd.cut(mlb_ml_bets.Juice_Home, bins=juice_bins, labels=False)

binarized_mlb_ml_bets = pd.get_dummies(mlb_ml_bets, columns=make_dummies)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [11]:
binarized_mlb_ml_bets.shape

(9173, 113)

In [12]:
features = binarized_mlb_ml_bets.columns.difference(['League', 'GameTime','Juice_Away', 'Juice_Home', 'MLWinner', 'Home', 'Away'])

target = ['MLWinner']

# Split into train and test data.
train_data = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime < '2017-07-01')][features]
train_labels = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime < '2017-07-01')][target]

test_data = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime >= '2017-07-01')][features]
test_labels = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime >= '2017-07-01')][target]

In [13]:
print (train_data.shape, test_data.shape)

(7945, 106) (1228, 106)


In [16]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 30, random_state = 42)

# Train the model on training data
rf.fit(train_data, train_labels.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [17]:
print('train: ', accuracy_score(rf.predict(train_data),train_labels))
print('test: ', accuracy_score(rf.predict(test_data),test_labels))

test_predicted_labels = rf.predict(test_data)
test_predicted_probs = rf.predict_proba(test_data)
print("\nConfusion matrix:\n%s" % confusion_matrix(test_labels, test_predicted_labels))
print("\nClassification report for classifier %s:\n%s\n"
% (rf, classification_report(test_labels, test_predicted_labels))) 

train:  0.898930144745
test:  0.551302931596

Confusion matrix:
[[283 284]
 [267 394]]

Classification report for classifier RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False):
             precision    recall  f1-score   support

       Away       0.51      0.50      0.51       567
       Home       0.58      0.60      0.59       661

avg / total       0.55      0.55      0.55      1228




In [18]:
# Saving feature names for later use
feature_list = list(train_data.columns)

# Convert to numpy array
features = np.array(features)

In [19]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:30} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: HC22_Away                      Importance: 0.0336
Variable: HC22_Home                      Importance: 0.0328
Variable: HC9_Home                       Importance: 0.0326
Variable: HC26_Away                      Importance: 0.0296
Variable: HC15_Away                      Importance: 0.0255
Variable: HC1_Away                       Importance: 0.0244
Variable: HC24_Home                      Importance: 0.0241
Variable: HC13_Home                      Importance: 0.0226
Variable: HC11_Home                      Importance: 0.0212
Variable: HC1_Home                       Importance: 0.0199
Variable: HC30_Home                      Importance: 0.0196
Variable: HC12_Away                      Importance: 0.0195
Variable: HC28_Home                      Importance: 0.0192
Variable: HC17_Away                      Importance: 0.019
Variable: HC26_Home                      Importance: 0.0187
Variable: HC11_Away                      Importance: 0.018
Variable: HC3_Home                       I

#### Model 2 - Include home and away team information as well

In [20]:
juice_bins = [-9999, -300, -200, -190, -180, -170, -160, -150, -140, -130, -120, -110, -100, 
             100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 300, 9999]

make_dummies = ['Binned_Juice_Away', 'Binned_Juice_Home', 'Home', 'Away',
           'HC1', 'HC2', 'HC3', 'HC4', 'HC5', 'HC6', 'HC7', 'HC8', 'HC9', 'HC10',
           'HC11', 'HC12', 'HC13', 'HC14', 'HC15', 'HC16', 'HC17', 'HC18', 'HC19', 'HC20', 
           'HC21', 'HC22', 'HC23', 'HC24', 'HC25', 'HC26', 'HC27', 'HC28', 'HC29', 'HC30']

mlb_ml_bets['Binned_Juice_Away'] = pd.cut(mlb_ml_bets.Juice_Away, bins=juice_bins, labels=False)
mlb_ml_bets['Binned_Juice_Home'] = pd.cut(mlb_ml_bets.Juice_Home, bins=juice_bins, labels=False)

binarized_mlb_ml_bets = pd.get_dummies(mlb_ml_bets, columns=make_dummies)

features = binarized_mlb_ml_bets.columns.difference(['League', 'GameTime','Juice_Away', 'Juice_Home', 'MLWinner'])

target = ['MLWinner']

# Split into train and test data.
train_data = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime < '2017-07-01')][features]
train_labels = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime < '2017-07-01')][target]

test_data = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime >= '2017-07-01')][features]
test_labels = binarized_mlb_ml_bets[(binarized_mlb_ml_bets.GameTime >= '2017-07-01')][target]

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 30, random_state = 42)

# Train the model on training data
rf.fit(train_data, train_labels.values.ravel())

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [21]:
print('train: ', accuracy_score(rf.predict(train_data),train_labels))
print('test: ', accuracy_score(rf.predict(test_data),test_labels))

test_predicted_labels = rf.predict(test_data)
test_predicted_probs = rf.predict_proba(test_data)
print("\nConfusion matrix:\n%s" % confusion_matrix(test_labels, test_predicted_labels))
print("\nClassification report for classifier %s:\n%s\n"
% (rf, classification_report(test_labels, test_predicted_labels))) 

train:  0.996727501573
test:  0.542345276873

Confusion matrix:
[[296 271]
 [291 370]]

Classification report for classifier RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False):
             precision    recall  f1-score   support

       Away       0.50      0.52      0.51       567
       Home       0.58      0.56      0.57       661

avg / total       0.54      0.54      0.54      1228




In [22]:
# Saving feature names for later use
feature_list = list(train_data.columns)

# Convert to numpy array
features = np.array(features)

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:30} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: HC9_Home                       Importance: 0.0163
Variable: HC22_Home                      Importance: 0.0161
Variable: HC22_Away                      Importance: 0.0155
Variable: HC26_Away                      Importance: 0.0154
Variable: HC15_Away                      Importance: 0.0141
Variable: HC24_Home                      Importance: 0.0134
Variable: HC1_Away                       Importance: 0.0118
Variable: Binned_Juice_Home_12           Importance: 0.0116
Variable: HC13_Home                      Importance: 0.0112
Variable: HC30_Home                      Importance: 0.011
Variable: HC12_Away                      Importance: 0.0108
Variable: HC9_Away                       Importance: 0.0108
Variable: HC26_Home                      Importance: 0.0106
Variable: Binned_Juice_Home_10           Importance: 0.0103
Variable: HC11_Home                      Importance: 0.0102
Variable: HC1_Home                       Importance: 0.0101
Variable: Binned_Juice_Home_9            